In [53]:
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.tools import BaseTool
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.agents import initialize_agent
from langchain.embeddings import OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory, ChatMessageHistory
from langchain.vectorstores  import Chroma
from langchain.agents import Tool, tool
import langchain
from langchain.agents import AgentType
from langchain.callbacks import get_openai_callback

from dotenv import load_dotenv, find_dotenv
import openai, os
_ = load_dotenv(find_dotenv())
openai.api_key = os.getenv('OPENAI_API_KEY')


In [54]:
persist_directory = "dbChroma"
vectordb = Chroma(
    collection_name="serprolim_collection",
    embedding_function= OpenAIEmbeddings(),
    persist_directory=persist_directory
    )


Using embedded DuckDB with persistence: data will be stored in: dbChroma


In [75]:
question = "Que profundidad debe tener un pozo?"
docs = vectordb.similarity_search(question,k=2)
docs

[Document(page_content='La profundidad del pozo séptico dependerá de la profundidad a la cual se encuentre la capa de arena, en Santa Cruz, por lo general se encuentra a partir de los tres metros. Lo ideal es que su pozo séptico pueda atravesar por lo menos un metro en la capa de arena y si existiese una corriente de agua subterránea, que exista al menos un metro y medio entre ella y la base del pozo.', metadata={'source': '6-pl'}),
 Document(page_content='La duración de un pozo séptico depende de su tamaño y del número de usuarios a los cuales atiende. Una cámara séptica bien construida, almacenará normalmente sólidos que deberán limpierse periodicamente. Un pozo deberia tener una vida util entre 3 a 12 años.', metadata={'source': '7-pl'})]

In [76]:
# chat completion llm
chat=ChatOpenAI(
    temperature=0.1,
    model_name='gpt-3.5-turbo',
    )

# conversational memory
# memory = ConversationBufferMemory(
#     memory_key="chat_history", 
#     return_messages=True
#     )

# # retrieval augmented pipeline for chatbot
# qa = RetrievalQA.from_chain_type(
#     llm=chat,
#     chain_type="stuff",
#     retriever = vectordb.as_retriever(search_kwargs={"k":2})
# )

chat.predict("Que profundidad debe terner un pozo?")

In [77]:
from langchain.prompts import PromptTemplate
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)


'La profundidad de un pozo puede variar dependiendo de su propósito y ubicación. Algunos factores a considerar son:\n\n1. Uso del pozo: Si el pozo se utiliza para abastecer agua potable, la profundidad debe ser suficiente para alcanzar un acuífero subterráneo que proporcione agua de calidad. Esto puede variar según la región y la disponibilidad de agua subterránea.\n\n2. Ubicación geográfica: La profundidad del pozo también puede depender de la ubicación geográfica y las características del suelo. En algunas áreas, el agua subterránea puede estar más cerca de la superficie, mientras que en otras puede estar a mayor profundidad.\n\n3. Tipo de suelo: El tipo de suelo también puede influir en la profundidad del pozo. Algunos suelos pueden retener más agua, lo que puede requerir una menor profundidad del pozo, mientras que otros suelos pueden ser más permeables y requerir una mayor profundidad para alcanzar el agua subterránea.\n\nEn general, los pozos de agua potable suelen tener una prof

In [59]:
def get_context(str,k):
    retriever = vectordb.as_retriever(search_kwargs={"k":k})
    context = retriever.get_relevant_documents(str)
    page_contents = [item.page_content for item in context]
    combined_content = ". ".join(page_contents)
    # Imprimir el resultado
    # print(combined_content)
    return combined_content

In [69]:
from langchain import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    HumanMessage,
)
query = "Que servicios dan?"

context = get_context(query,2)

template  = """
Eres un asistente de atención al cliente para una empresa especializada en la limpieza de pozos y cámaras sépticas (pozos ciegos). \
Utiliza los siguientes fragmentos de contexto para responder la pregunta al final. \
Si no sabes la respuesta, simplemente di que no lo sabes, NO intentes inventar una respuesta. \
Utiliza un máximo de dos oraciones. Mantén la respuesta lo más concisa posible.

{context}
Question: {question}
Helpful Answer:"""

QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
    )

# ai_template = AIMessagePromptTemplate.from_template("{response}")

messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template(human_template)
    ]

chat_prompt = ChatPromptTemplate.from_messages( messages )

resp = chat([HumanMessage(content=query)])


In [71]:
chat_prompt

ChatPromptTemplate(input_variables=['input'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template='\nEres un asistente de atención al cliente para una empresa especializada en la limpieza de pozos sépticos y cámaras sépticas (pozos ciegos).     Toda la información necesaria está contenida en el contexto delimitado por los signos de triple igualdad.     Por favor, abstente de utilizar recursos externos, enlaces o referencias que no estén presentes en el contexto.     No te basarás en tu propio conocimiento sobre el tema. Si la respuesta no se encuentra en el contexto,     simplemente responde con "Hmm, no estoy seguro/a". No intentes inventar una respuesta.\n===\nDamos el servicio de limpieza de pozos sépticos, pozos ciegos y cámaras sépticas.. Podemos mejorar el precio en función a la frecuencia que con la que nos solicite el servicio y la cantidad de servicios que p

In [70]:
print (resp)




content='Como modelo de lenguaje de IA, no proporciono servicios directamente. Sin embargo, puedo ayudarte a responder preguntas, proporcionar información y asistirte en diversas tareas. Algunos ejemplos de servicios que puedo ofrecer son:\n\n- Responder preguntas generales sobre una ampl' additional_kwargs={} example=False


In [ ]:
# format with required inputs
chat_prompt_value = chat_prompt.format_prompt(
    context=combined_content,
    input="Como te llamas?",
)
chat_prompt_value

In [ ]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)
messages = chat_prompt_value.to_messages()

messages.append(
    HumanMessage(content="Que profundidad debe tener el pozo?")
)

res = chat(messages)

print(f"Length: {len(res.content)}\n{res.content}")

In [ ]:
class PrecioTool(BaseTool):
    name = "Calcula el precio"
    description = "Useful when you need to answer questions about prices or the cost of the service."

    def _run(self, ubicacion: str) -> str:
        return "Bs.300"
    
    def _arun(self, ubicacion: str) -> str:
        # return "Bs.400"
        raise NotImplementedError("This tool does not support async")
    
# @tool
# def reverse_string(query: str) -> str:
#     '''Reverses a string.'''
#     return query[::-1]

In [ ]:
# retrieval qa chain
qa = ConversationalRetrievalChain.from_llm(llm=llm,
                                 retriever=retriever,
                                 chain_type="stuff",
                                 verbose=True,
                                 memory=memory,
                                 combine_docs_chain_kwargs={'prompt': qa_prompt}
                                 )

In [ ]:
tools = [
    PrecioTool(),
    Tool(
        name="Knowledge Base",
        func=qa.run,
        description="Useful when you need to answer general knowledge. If you don't know the answer, just say that you don't know, don't try to make up an answer.",
    ),
    # reverse_string
    ]

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# initialize agent with tools
agent = initialize_agent(
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=2,
    early_stopping_method='generate',
    memory=memory,
    # handle_parsing_errors=True,
)

In [ ]:
agent.agent.llm_chain.verbose=True
langchain.debug = False

In [ ]:
with get_openai_callback() as cb:
    response = agent.run("Que profundidad debe tener un pozo?")
    print(response)
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")

# response = agent.run("Cyuanto cuesta el serivio?")

In [ ]:
print(response)

In [ ]:

agent.agent.llm_chain.prompt